In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision import models

In [ ]:
# import pandas as pd
# import torch
# from torchvision import transforms

# # read the Excel file into a pandas dataframe
# data = pd.read_excel('/content/final.xlsx')

# # extract the image drive link and target values from the dataframe
# image_links = data.iloc[:, 0].tolist()
# targets = data.iloc[:, 1:3].values

# # define a transformation to apply to the images
# transform = transforms.Compose([
#     transforms.Resize((512, 512)),  # resize the image to 224x224
#     transforms.ToTensor()  # convert the image to a PyTorch tensor
# ])

In [ ]:
image_links[0]

'/content/drive/MyDrive/btp/day3-01.png'

In [ ]:
import pandas as pd
import torch
from torchvision import transforms
import numpy as np

from sklearn.preprocessing import MinMaxScaler

# read the Excel file into a pandas dataframe
data = pd.read_excel('/content/btp.xlsx')

# extract the image drive link and target values from the dataframe
image_links = data.iloc[:, 0].tolist()
image_links = [i[:-3] + 'png' for i in image_links]
targets = data.iloc[:, 1].values
targets2 = data.iloc[:, 2].values
print(targets)
print(targets2)
minT2 =  np.min(targets2)
maxT2 = np.max(targets2)

minT1 =  np.min(targets)
maxT1 = np.max(targets)

'conte'
# Min-Max normalization
# targets2 = (targets2 - minT2) / (maxT2 -minT2)
# targets = (targets -minT1) / (maxT1 - minT1)

# define a transformation to apply to the images
transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor()  # convert the image to a PyTorch tensor
])

[2 2 2 3 3 2 3 2 2 3 2 3 2 3 2 3 4 5 4 3 4 2 5 4 5 4 0 5 5 5 5 5 5 5 5 5 5
 4 2 4 4 4 4 0 4 4 4 4 5 0]
[2 2 2 3 3 2 3 3 3 3 2 3 2 4 4 4 4 4 4 4 5 4 5 5 5 5 0 4 4 4 3 4 5 5 4 5 5
 4 3 4 4 4 3 2 3 4 4 3 5 3]


In [ ]:
from PIL import Image


In [ ]:
# create a dataset object
class MyDataset(torch.utils.data.Dataset):
    def  __init__(self, image_links, targets, target2, transform):
        self.image_links = image_links
        self.targets = targets
        self.targets2 = targets2
        self.transform = transform

    def __len__(self):
        return len(self.image_links)

    def __getitem__(self, idx):
        # load the image from the drive link
        image = Image.open(self.image_links[idx]).convert('RGB')

        # apply the transformation to the image
        image = self.transform(image)

        # get the target values for the image
        target = torch.tensor([self.targets[idx],self.targets2[idx]], dtype=torch.float32)
        # print(self.targets2[idx])

        # target2 = torch.tensor(float(self.targets2[idx]), dtype=torch.float32)

        # print(target.size())
        # print(target2.size())
        # print(target2)


        # concatenated_target = torch.cat((target, target2.view(1)), dim=0)

        return image,   target




# create a dataset object from the data
dataset = MyDataset(image_links, targets,targets2, transform)

from torch.utils.data import random_split

# Assuming you have a complete dataset named 'dataset'
dataset_size = len(dataset)
train_size = int(0.8 * dataset_size)
val_size = dataset_size - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])


# create a dataloader object to load the data in batches
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True)
valloader = torch.utils.data.DataLoader(val_dataset, batch_size=4, shuffle=True)

In [ ]:
for i in trainloader:
  print(i)
  break

[tensor([[[[0.9804, 0.9569, 0.9608,  ..., 0.9961, 0.9961, 1.0000],
          [0.9216, 0.8745, 0.8902,  ..., 0.9804, 0.9765, 0.9882],
          [0.9412, 0.8980, 0.9098,  ..., 1.0000, 0.9922, 0.9922],
          ...,
          [0.9922, 0.9922, 1.0000,  ..., 1.0000, 0.9922, 0.9922],
          [0.9882, 0.9765, 0.9765,  ..., 0.9804, 0.9765, 0.9882],
          [1.0000, 0.9961, 0.9961,  ..., 0.9961, 0.9961, 1.0000]],

         [[0.9804, 0.9569, 0.9608,  ..., 0.9961, 0.9961, 1.0000],
          [0.9216, 0.8745, 0.8902,  ..., 0.9804, 0.9765, 0.9882],
          [0.9412, 0.8980, 0.9098,  ..., 1.0000, 0.9922, 0.9922],
          ...,
          [0.9922, 0.9922, 1.0000,  ..., 1.0000, 0.9922, 0.9922],
          [0.9882, 0.9765, 0.9765,  ..., 0.9804, 0.9765, 0.9882],
          [1.0000, 0.9961, 0.9961,  ..., 0.9961, 0.9961, 1.0000]],

         [[0.9804, 0.9569, 0.9608,  ..., 0.9961, 0.9961, 1.0000],
          [0.9216, 0.8745, 0.8902,  ..., 0.9804, 0.9765, 0.9882],
          [0.9412, 0.8980, 0.9098,  ..., 

In [ ]:

# Define the device to use for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
# Define the CNN
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.cnn = models.resnet18(pretrained=True)
        self.cnn.fc = nn.Identity()

    def forward(self, x):
        x = self.cnn(x)
        return x

# Define the FCN
class FCN(nn.Module):
    def __init__(self):
        super(FCN, self).__init__()
        self.fc1 = nn.Linear(512, 256)
        self.fc2 = nn.Linear(256, 2)

    def forward(self, x):
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Define the combined model
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.cnn = CNN()
        self.fcn = FCN()

    def forward(self, x):
        x = self.cnn(x)
        x = self.fcn(x)
        return x

# Define the loss function
criterion = nn.MSELoss()

# Define the optimizer
model = Model()
model = model.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 74.4MB/s]


In [ ]:
num_epochs=50

In [ ]:

# Train the model
for epoch in range(num_epochs):
    for images, labels in trainloader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        # print(outputs.shape)
        # print(labels.shape)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        # Print the loss and accuracy for this batch
        with torch.no_grad():
            predictions = torch.round(outputs)
            accuracy = torch.mean((predictions == labels).float())
            print("Epoch [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%".format(epoch+1, num_epochs, loss.item(), accuracy.item() * 100))


In [ ]:
image = Image.open('/content/drive/MyDrive/btp/day14-03.png').convert('RGB')

# apply the transformation to the image
image = transform(image)
image=image.unsqueeze(1).view(1,3,512,512)


outputs = model(image.to(device))


In [ ]:
outputs

tensor([[3.9199, 3.8013]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [ ]:
# Custom loss function for area prediction
class AreaLoss(nn.Module):
    def __init__(self):
        super(AreaLoss, self).__init__()

    def forward(self, predicted_area, ground_truth_area):
        loss = torch.mean((predicted_area - ground_truth_area) ** 2)
        return loss


In [ ]:
# Define the CNN
class CNNarea(nn.Module):
    def __init__(self):
        super(CNNarea, self).__init__()
        self.cnn = models.resnet18(pretrained=True)
        self.cnn.fc = nn.Identity()

    def forward(self, x):
        x = self.cnn(x)
        return x

# Define the FCN
class FCNarea(nn.Module):
    def __init__(self):
        super(FCNarea, self).__init__()
        self.fc1 = nn.Linear(512, 256)
        self.fc2 = nn.Linear(256, 1)

    def forward(self, x):
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Define the combined model
class Modelarea(nn.Module):
    def __init__(self):
        super(Modelarea, self).__init__()
        self.cnn = CNNarea()
        self.fcn = FCNarea()

    def forward(self, x):
        x = self.cnn(x)
        x = self.fcn(x)
        return x

# Define the loss function
criterion = AreaLoss()

# Define the optimizer
modelarea = Modelarea()
modelarea = modelarea.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
num_epochs=50

In [ ]:

# Train the model
for epoch in range(num_epochs):
    for images, _,labels ,in trainloader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = modelarea(images)
        print(outputs.shape)
        print(labels.shape)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        # Print the loss and accuracy for this batch
        with torch.no_grad():
            predictions = torch.round(outputs)
            accuracy = torch.mean((predictions == labels).float())
            print("Epoch [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%".format(epoch+1, num_epochs, loss.item(), accuracy.item() * 100))
